In [8]:
# 종목 선별
'''
8시 50분에 3%이상친 종목 선별

그중에 거래대금(turnover) 순으로 최대 100개만
'''
import pybitClass

cl = pybitClass.BybitAPI()

df = cl.all_tickers()
df = df.sort_values('turnover24h', ascending=False).reset_index(drop = True)

df = df[df["turnover24h"] > 4e06][:100]
df

,symbol,lastPrice,indexPrice,markPrice,prevPrice24h,price24hPcnt,highPrice24h,lowPrice24h,prevPrice1h,openInterest,openInterestValue,turnover24h,volume24h,fundingRate,nextFundingTime,predictedDeliveryPrice,basisRate,deliveryFeeRate,deliveryTime,ask1Size,bid1Price,ask1Price,bid1Size,basis,preOpenPrice,preQty,curPreListingPhase
0,BTCUSDT,62889.000000,62922.480000,62889.080000,63623.000000,-0.011536,63845.000000,62464.400000,62913.900000,6.543965e+04,4.115439e+09,4.822479e+09,7.641575e+04,-0.000058,1.719907e+12,,,,0.0,2.376,62888.900000,62889.000000,10.86,,,,
1,ETHUSDT,3446.640000,3448.360000,3446.620000,3517.010000,-0.020008,3523.570000,3419.280000,3443.670000,7.053276e+05,2.430996e+09,1.434270e+09,4.131203e+05,-0.000011,1.719907e+12,,,,0.0,0.220,3446.640000,3446.650000,107.02,,,,
2,SOLUSDT,147.690000,147.750000,147.690000,148.180000,-0.003306,149.290000,145.800000,147.570000,3.762008e+06,5.556110e+08,7.251945e+08,4.917868e+06,0.000006,1.719907e+12,,,,0.0,63.500,147.680000,147.690000,319.70,,,,
3,1000PEPEUSDT,0.011416,0.011419,0.011416,0.012116,-0.057784,0.012145,0.011306,0.011452,1.726462e+10,1.970877e+08,6.436843e+08,5.575178e+10,0.000100,1.719907e+12,,,,0.0,22000.000,0.011416,0.011416,33000.00,,,,
4,WIFUSDT,2.298800,2.300800,2.300200,2.274000,0.010905,2.356600,2.163100,2.279300,4.372672e+07,1.005802e+08,5.569754e+08,2.476870e+08,0.000100,1.719907e+12,,,,0.0,53.000,2.298800,2.298900,554.00,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,WAVESUSDT,1.044000,1.045100,1.044400,1.066500,-0.021097,1.088500,1.033000,1.069500,6.772171e+06,7.072855e+06,1.082526e+07,1.018036e+07,0.000100,1.719907e+12,,,,0.0,772.200,1.044000,1.044500,1610.20,,,,
96,10000SATSUSDT,0.001830,0.001830,0.001830,0.001969,-0.070594,0.001970,0.001789,0.001829,2.541892e+09,4.651662e+06,1.072622e+07,5.814203e+09,0.000100,1.719907e+12,,,,0.0,1595600.000,0.001830,0.001831,610000.00,,,,
97,ETHFIUSDT,2.840000,2.842000,2.840000,3.179000,-0.106637,3.198000,2.825000,2.865000,4.143739e+06,1.176822e+07,1.065310e+07,3.540421e+06,-0.000121,1.719907e+12,,,,0.0,618.400,2.839000,2.840000,227.20,,,,
98,GMEUSDT,0.006917,0.006904,0.006917,0.008500,-0.186235,0.008555,0.006806,0.006990,4.114254e+08,2.845830e+06,1.050861e+07,1.412045e+09,0.000100,1.719907e+12,,,,0.0,18450.000,0.006917,0.006920,2970.00,,,,


In [9]:
'''
08:50에 종목선별 -> 59분에 타겟 서치 -> 타겟 알림 -> 매수 원하는 종목 전달 -> 지정가로 종가에 매수
-> 매수 되면 알림(하루동안 주문 안되면 주문취소 처리)-> 2음봉뜨면 스위칭 or 3배기준 3% 이상이면 익절

'''
import datetime
import pybitClass
import time
import indicators as idt
import pandas as pd

cl = pybitClass.BybitAPI()

# 종목 선별
def select_tickers():
    return cl.all_tickers()[df['change']>3].sort_values('turnover24h', ascending=False).reset_index(drop = True)[:100]

# 타겟 선별
def select_targets(tickers):
    targets = pd.DataFrame()
    for idx, i in enumerate(tickers):
        print(f'target함수 for문 확인용 : {idx,i}')
        df  = cl.klines(i, 'D', 100)
        o,h,l,c,v,ch = df,open, df.high, df.low, df.close, df.volume, df.change
        bb20 = idt.Bollinger_Band(df,20,2)
        if (c.iloc[-6:-1]<bb20.iloc[-6:-1])and(o.iloc[-6:-1]<bb20.iloc[-6:-1])and((c.iloc[-1]>bb20.iloc[-1]))and(ch.iloc[-1]>3):
            targets.append({'종목':i, '등락율':ch.iloc[-1]})
    return targets

# 손익 확인
def pnl():
    posi = cl.position_info(settleCoin="USDT")
    if not posi.empty:
        entry = float(posi["avgPrice"][0])
        mark = float(posi["markPrice"][0])
        leverage = float(posi["leverage"][0])
        posi["pnl"] = round((((mark - entry) / mark) * 100 * leverage), 2)
        posi.loc[posi["side"] == "Sell", "pnl"] = -posi.loc[
            posi["side"] == "Sell", "pnl"
        ]
        posi = posi[["symbol", "pnl"]]
        asset = cl.wallet()
        posi = posi.join(asset)
        return posi.transpose()
    return pd.DataFrame()

# 마진타입 레버리지 세팅
def set_margin_leverage(targets, leverage):
    for i in targets.종목:
        cl.set_marginType(i, 1, leverage)
        cl.set_leverage(i, leverage)

while 1:
    now = datetime.datetime.now()

    if now.hour==8 and now.minute==50:
        # 종목 선별
        tickers = select_tickers()
        time.sleep(60)

    if now.hour==8 and now.minute==58:
        # 손,익 확인
        pnl()

    if now.hour==8 and now.minute==59:
        # 타겟 선별
        targets = select_targets()
        # 마진타입, 레버리지 세팅
        set_margin_leverage(targets,3)